In [1]:
# hide
# default_exp utils

# Sciflow utils

In [2]:
# export

import os
from pathlib import Path

import nbdev
from fastcore.script import call_parse
from nbdev.clean import nbdev_clean_nbs
from nbqa.find_root import find_project_root
from turbodbc import Megabytes, connect, make_options

In [3]:
%load_ext autoreload
%autoreload 2

# Paths

In [4]:
# export


def lib_path(*lib_relative_path):
    lib_root_path = find_project_root(srcs=(str(Path(".").resolve()),))
    return Path(os.path.join(lib_root_path, *lib_relative_path))

In [5]:
assert str(lib_path("nbs")).endswith("sciflow/nbs")
assert Path("test/test_clustering.ipynb").resolve() == lib_path(
    "nbs", "test", "test_clustering.ipynb"
)

# Sciflow Clean

In [6]:
# export


@call_parse
def sciflow_clean():
    nbdev.clean.cell_metadata_keep = nbdev.clean.cell_metadata_keep + ["tags"]
    nbdev_clean_nbs()

# ODBC Connection

In [7]:
# export


def prepare_env(env_file_path: str = None):
    if env_file_path is None:
        env_file_path = os.path.expanduser("~/.sciflow/env")
    # TODO create this for user
    if not os.path.exists(env_file_path):
        raise EnvironmentError(
            f"You need to create a Sciflow environment vars file at: {env_file_path}"
        )
    with (open(env_file_path, "r")) as env_file:
        for line in env_file.readlines():
            key, value = line.strip().split("=", 1)
            os.environ[key.replace("export ", "")] = value

In [8]:
# export


def odbc_connect(options=None, env_file_path: str = None):
    required_vars = ("ODBC_DRIVER", "ODBC_HOST", "ODBC_PORT", "ODBC_USER", "ODBC_PWD")
    if not all([v in os.environ for v in required_vars]):
        prepare_env(env_file_path)
    if options is None:
        options = make_options(
            fetch_wchar_as_char=True, autocommit=True, read_buffer_size=Megabytes(5),
        )
    connection = connect(
        driver=os.environ["ODBC_DRIVER"],
        host=os.environ["ODBC_HOST"],
        port=os.environ["ODBC_PORT"],
        uid=os.environ["ODBC_USER"],
        pwd=os.environ["ODBC_PWD"],
        SSL=1,
        TrustedCerts=os.environ["SSL_CERTS"],
        turbodbc_options=options,
    )
    return connection

In [9]:
# export


def query(
    conn,
    sql,
    strings_as_dictionary=False,
    adaptive_integers=False,
    strings_to_categorical=False,
    date_as_object=False,
):
    with conn.cursor() as cursor:
        df = (
            cursor.execute(sql)
            .fetchallarrow(strings_as_dictionary=False, adaptive_integers=False,)
            .to_pandas(strings_to_categorical=False, date_as_object=False,)
        )
    return df

In [10]:
try:
    del os.environ["ODBC_DRIVER"]
    del os.environ["ODBC_HOST"]
    del os.environ["ODBC_PORT"]
    del os.environ["ODBC_USER"]
    del os.environ["ODBC_PWD"]
    del os.environ["SSL_CERTS"]
except KeyError:
    pass

In [11]:
%%time
import turbodbc

conn = odbc_connect()

CPU times: user 16 ms, sys: 8 ms, total: 24 ms
Wall time: 343 ms


In [12]:
%%time
assert type(conn) == turbodbc.connection.Connection
assert query(conn, "SELECT 1 AS test_col")["test_col"].iloc[0] == 1

CPU times: user 8 ms, sys: 4 ms, total: 12 ms
Wall time: 289 ms
